## SBBGCA usage
Usage of SBBGCA.ipynb

### Set Variable 
set variables before running codes:
data_directory, tag, outputPath

### Want to get clustering results
##### run startClustering() function.

It call do_SBBGCA_clustering() function in SBBGAC.ipynb which output 2 results : intermediatePool, initialNames.

##### intermediatePool: the intermediates of clustering which is a dict
##### initialNames: a dict which contains the original names of basic group(start from G0 to G_inputHooklogNumber)

in the startClustering() function will output 2 pickles to outputPath which helping further usage.
(Because the performance of SBBGCA is terrible Q_Q)

### Want to draw hierarchy graph of clustering result
##### run drawClusteringResults() function.

It read the pickles of clustering results and draw graphs using matplotlib.

In [1]:
import os
import numpy as np
import glob
import shutil
import csv
import datetime
import threading
import time
import math
import traceback
import pickle
from multiprocessing import *

# data preprocessing
## to move too large file to outlier folder (mean+2*std=95%)
### dict generated for automatic run
* for python3.6
* test succeed by ubuntu16.04

*add by leoqaz12@20180414*

* 修改featureTree 以及 RasMMa 檔案以符合python3.6規範，並減低出錯率
* preprocessing部分會將太大的hooklog踢去outlier資料夾，減低出錯率

In [2]:
allPath=[]
for dirpath, dirnames, filenames in os.walk('../data/train_17family/'):  #要改
    if dirpath.split('/')[-2] != 'train_17family' :  #要改
        allPath.append(dirpath)
        print(dirpath)

../data/train_17family/madang/child
../data/train_17family/madang/child/outlier
../data/train_17family/madang/main
../data/train_17family/madang/main/outlier
../data/train_17family/madang/4 child
../data/train_17family/madang/3 child
../data/train_17family/soltern/child
../data/train_17family/soltern/main
../data/train_17family/soltern/main/outlier
../data/train_17family/soltern/3 child
../data/train_17family/somoto/child
../data/train_17family/somoto/main
../data/train_17family/somoto/main/outlier
../data/train_17family/somoto/3 child
../data/train_17family/somoto/3 child/outlier
../data/train_17family/elkern/child
../data/train_17family/elkern/child/outlier
../data/train_17family/elkern/main
../data/train_17family/elkern/main/outlier
../data/train_17family/elkern/4 child
../data/train_17family/elkern/3 child
../data/train_17family/ramnit/5 child
../data/train_17family/ramnit/child
../data/train_17family/ramnit/child/outlier
../data/train_17family/ramnit/6 child
../data/train_17family

In [3]:
# my_dir = r'../data/train_17family/graftor/main/'
filesize=[]
lastname = 'None'
dicts = {}
for i,my_dir in enumerate(allPath):
    name = my_dir.split('/')[-2]
    if name == lastname:
        for f in os.listdir(my_dir):
            path = os.path.join(my_dir, f)
            if os.path.isfile(path):
                filesize.append(os.path.getsize(path))
#                 print (os.path.getsize(path))
    elif i>0:
        last = np.array(filesize)
        upper = last.mean()+2*last.std()
        dicts[lastname]=upper
        lastname = name
        filesize=[]
        for f in os.listdir(my_dir):
            path = os.path.join(my_dir, f)
            if os.path.isfile(path):
                filesize.append(os.path.getsize(path))
    else:
        lastname = name
        for f in os.listdir(my_dir):
            path = os.path.join(my_dir, f)
            if os.path.isfile(path):
                filesize.append(os.path.getsize(path)) 
last = np.array(filesize)
upper = last.mean()+2*last.std()
dicts[lastname]=upper
dicts

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:15: RuntimeWarning: Mean of empty slice.
/home/leoqaz12/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/leoqaz12/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/leoqaz12/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/leoqaz12/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/leoqaz12/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:30: RuntimeWarning: Mean of empty slice.


{'madang': 29254.0,
 'child': 271485.6949261952,
 'main': 303608.0,
 'soltern': nan,
 'somoto': 224290.3366994316,
 '3 child': 232659.6272698663,
 'elkern': 22163.14621611852,
 'ramnit': 22514.398425986652,
 '4 child': 12971.0,
 'eggnog': 96356.12245999707,
 'mira': 22447.26167179696,
 'strictor': 35811.0,
 'ibryte': nan,
 'autoit': 92795.71296134185,
 'parite': 41323.57799255119,
 'firseria': 17967.475142368126,
 'graftor': 9900.50720052833,
 'solimba': 160963.45638502305,
 'shodi': 87393.24309294473,
 'bettersurf': 79381.71376914793,
 'lydra': nan}

In [4]:
for k,v in dicts.copy().items():
    if v>1500000:
        print(k,v)
        dicts[k]=1500000
    if math.isnan(v):
        print(k,v)
        del dicts[k]
#         dicts[k]

soltern nan
ibryte nan
lydra nan


In [5]:
# for verify
for k,v in dicts.items():
    if v>1500000:
        print(k,v)
#         dicts[k]=1500000
    if math.isnan(v):
        print(k,v)
#         del dicts[k]
#         dicts[k]

In [6]:
# len(dicts)
# v
# for k in dicts:
#     print(k)
# for v in glob.glob('../data/train_17family/'+k+'/*'):
#     print(v)
# v.split('/')[-1]
# kk = dicts['hamhuf'] 
# upper
# math.isnan(kk)

In [7]:
count=0
for k in dicts:
    for v in glob.glob('../data/train_17family/'+k+'/*'):  #k所有家族名稱 change
        for f in os.listdir(v):  #v=>所有代別, f 所有hooklogs檔案名稱
            path = os.path.join(v, f)
            if os.path.isfile(path) and os.path.getsize(path)>dicts[k]:
                if not os.path.isdir(v+'/outlier'):
                    os.makedirs(v+'/outlier')
                shutil.move(path,v+'/outlier/')
                print(path,v+'/outlier/')
                count+=1
#         dicts[v.split('/')[-2]+v.split('/')[-1]]=count
        print('=====================',v.split('/')[-2],v.split('/')[-1],count,'======================')
        count=0
#     print(k)

===================== madang child 0 ======================
===================== madang main 0 ======================
===================== madang 4 child 0 ======================
===================== madang 3 child 0 ======================
===================== somoto child 0 ======================
===================== somoto main 0 ======================
===================== somoto 3 child 0 ======================
===================== elkern child 0 ======================
===================== elkern main 0 ======================
===================== elkern 4 child 0 ======================
===================== elkern 3 child 0 ======================
===================== ramnit 5 child 0 ======================
===================== ramnit child 0 ======================
===================== ramnit 6 child 0 ======================
../data/train_17family/ramnit/main/826d9287ae211630f260f58c77e48e0b2381c367cc82b672819d3e86872e2e5a_3100.trace.hooklog ../data/train_17family/ramnit/

### store and reload dicts

In [34]:
with open('../data/dicts_901.pickle', 'wb') as handle:
    pickle.dump(dicts, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [2]:
with open('../data/dicts_901.pickle', 'rb') as handle:
    dicts = pickle.load(handle)
dicts #upper bound

{'abhw': 126684.39736527948,
 'addlyrics': 42335.0,
 'adgazelle': 251062.06876060125,
 'adload': 416162.0495560344,
 'agdgc': 1500000,
 'aira': 153000.75633132982,
 'airinstall': 24617.0,
 'alisa': 307051.0,
 'aliser': 307051.0,
 'allaple': 582331.3803463926,
 'alman': 302327.0059193162,
 'almanahe': 321056.7331766427,
 'alyak': 297115.0,
 'amonetize': 222869.41000323094,
 'android': 372098.0,
 'androidos': 372098.0,
 'androm': 216487.16530630804,
 'andromeda': 171658.276355611,
 'antiav': 18919.0,
 'ardamax': 14136.0,
 'aroc': 438612.7560848078,
 'arqpcaf': 267034.05455157085,
 'aspxor': 47899.87246662805,
 'aups': 660230.0,
 'autoit': 209482.3446669383,
 'avmh': 22555.192534341746,
 'awdm': 57221.478140378014,
 'awdn': 127348.27468185642,
 'badur': 659104.2631946781,
 'banker': 309937.9918993648,
 'banload': 5803.0,
 'barys': 331400.738743226,
 'bayk': 994678.0361386486,
 'bayrob': 524865.6679938445,
 'bdmj': 106756.01719938249,
 'beebone': 20737.0,
 'berbew': 34911.54809687576,
 'be

# Main part Start

In [8]:
# reverse clusterID by hooklogName

def findClusterID(nameDict, hooklogName):
    for key, value in nameDict.items():
        if(value == hooklogName):
            return key
    return hooklogName

In [9]:
def getInitialNameDict(initialDict):
    nameDict = dict()
    for key, value in initialDict.items():
        name = value[0]
        nameDict[key] = name
    return nameDict

In [10]:
# convert memberSet to List type

def getMemberList(memberSet, nameDict):
    memberList = [] #list()
    while(len(memberSet)>0):
        member = memberSet.pop()
        clusterID = findClusterID(nameDict, member)
        memberList.append(clusterID)
    return memberList

In [11]:
# z[0] = g1,  z[1] = g2,  z[2] = 高度
# Create structure Z

def createStructZ(intermediate_dict, nameDict):
    import numpy as np
    Z = np.zeros((len(intermediate_dict) ,4))
    
    intermediate_list = sorted(intermediate_dict.items(), key=lambda x:x[0])
    
    iterCounter = 0
    for item in intermediate_list:
        value = item[1] # get original dict value
        score = value[0]
        height = 1 - score # get cluster distance
        clusterName = value[1][0]
        memberSet = value[2] # members set
        memberList = getMemberList(memberSet, nameDict)
        print(clusterName, " : ", memberList)
        member1 = memberList[0][1::]
        member2 = memberList[1][1::]
        
        Z[iterCounter] = [member1, member2, height, len(memberList)] # set Z element
        iterCounter+=1
        
    return Z

In [12]:
def createLabelList(nameDict):
    dict_keys = list(nameDict.keys())
    dict_keys.sort(key=lambda tup: int(tup[1::] )) # sort keys by number in clusterName (i.e., '31' in 'G31')
    
    labelList = [] #list()
    for key in dict_keys:
        labelList.append( nameDict[key] )
    
    return labelList

In [13]:
wrong=[]
def handle_error(e):
    '''處理 child process 的錯誤'''
    traceback.print_exception(type(e), e, e.__traceback__)
    wrong.append((type(e), e, e.__traceback__))

In [14]:
# basic global inputs variable
manualThresholdNumber = 0.8 # defined the threshold of merge score
# parameters
# leo = dicts#['elkern','eggnog','mira','autoit','parite','shodi','lydra'] #想要自動化完成的家族名稱
datadir='../data/train_17family/' #前綴路徑，family所在的根路徑
outpath = '../data/output/RasMMA-train_17family/' #輸出資料夾根路徑
# delay=35 #依照CPU單核心執行力自己調整，噴error就調大
# threadNum=100 #CPU核心線程占用數量，delay越大可以設的越大，記憶體不夠者請注意此量

# def muti_process_RasMMA(familyName,generation,c):
def muti_process_RasMMA(args):
    familyName,generation,c = args
    print('===RasMMA{}%: {} {}({}) START==='.format(c,familyName,generation, os.getpid()))
#     start = time.time()
    data_directory = datadir+ familyName +"/"+ generation +"/" # traces dir

    tag = familyName + "_0.8" # used for pickle name
    outputPath = outpath + tag + "/" + generation + "/"
    pickleDir = outputPath + "pickle/"

    # Create Directories if didn't exist

    if not os.path.isdir(outputPath): os.makedirs(outputPath)
    if not os.path.isdir(pickleDir): os.makedirs(pickleDir)

    # do clustering and output two pickle files. (@_intermediate.pickle and @_nameDict.pickle)
    % run RasMMA.ipynb

    def startClustering(data_directory, tag, outputPath, thresholdValue=None):
        if not os.path.isdir(outputPath): os.makedirs(outputPath) # create output path Directory

        intermediatePool, initialDict, roundInfos, residual = do_SBBGCA_clustering(data_directory, tag, outputPath, thresholdValue)

        # saving intermediatePool as pickle file
        with open(pickleDir + tag + '_intermediate.pickle', 'wb') as handle:
            pickle.dump(intermediatePool, handle, protocol=pickle.HIGHEST_PROTOCOL)

        # saving initialNames dict as pickle file
        with open(pickleDir + tag + '_initialDict.pickle', 'wb') as handle:
            pickle.dump(initialDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

        # saving round information dict as pickle file
        with open(pickleDir + tag + '_roundInfos.pickle', 'wb') as handle:
            pickle.dump(roundInfos, handle, protocol=pickle.HIGHEST_PROTOCOL)

        if(residual is not None):
            # saving round information dict as pickle file
            with open(pickleDir + tag + '_residual.pickle', 'wb') as handle:
                pickle.dump(residual, handle, protocol=pickle.HIGHEST_PROTOCOL)



    # usage example of get clustering results


#     date_time = datetime.datetime.now()
#     print(date_time.strftime("%Y-%b-%d %H:%M"))

    startClustering(data_directory, tag, outputPath, manualThresholdNumber)

#     date_time = datetime.datetime.now()
#     print(date_time.strftime("%Y-%b-%d %H:%M"))
    print('===RasMMA: {} {} END==='.format(familyName,generation))
    
    
    
    
# threads = []
c=0

# with Pool(processes=cpu_count()-2) as p: #CPU核心數目，決定提升速度的倍率
#     print('Parent process {}.'.format(os.getpid()))  #爸爸pid，如果跑到抓狂要去linux砍掉的話
#     for k in dicts: #leo
#         familyName = k
#         c+=1
#         for v in glob.glob(datadir+familyName+'/*'):
#             generation = v.split('/')[-1]
#             p.apply_async(muti_process_RasMMA,args=(familyName,generation,(c/len(dicts))*100,),error_callback=handle_error) ##東西少的時候(不會吃滿資源)速度快


args=[]
for k in dicts: #leo
    familyName = k
    for v in glob.glob(datadir+familyName+'/*'):
        generation = v.split('/')[-1]
        temp = (familyName,generation,(c/len(dicts))*100)
        args.append(temp)
with Pool(processes=cpu_count()-2) as p: #CPU核心數目，決定提升速度的倍率
    print('Parent process {}.'.format(os.getpid()))  #爸爸pid，如果跑到抓狂要去linux砍掉的話
    p.map_async(muti_process_RasMMA,args,error_callback=handle_error)  #東西多的時候資源調度處理較好

    

    
    #             thread1 = myThread(c, familyName, generation)
    print('=============Joining all processes plz wait....=================')
    p.close()
    p.join()

print("All Done~ ^^ muti-process try by leoqaz12")

===RasMMA0.0%: madang child(8811) START===
===RasMMA0.0%: elkern main(8815) START===
===RasMMA0.0%: somoto child(8813) START===
===RasMMA0.0%: somoto 3 child(8814) START===
===RasMMA0.0%: elkern 3 child(8816) START===
===RasMMA0.0%: ramnit child(8817) START===
===RasMMA0.0%: madang 4 child(8812) START===
===RasMMA0.0%: eggnog main(8820) START===
===RasMMA0.0%: ramnit main(8818) START===
===RasMMA0.0%: ramnit 3 child(8819) START===


ImportError: libcgraph.so.6: cannot open shared object file: No such file or directory

Parent process 8798.
=============Joining all processes plz wait....=================


ImportError: libcgraph.so.6: cannot open shared object file: No such file or directory

ImportError: libcgraph.so.6: cannot open shared object file: No such file or directory

...

Round:  1
Round:  1


ImportError: libcgraph.so.6: cannot open shared object file: No such file or directory

ImportError: libcgraph.so.6: cannot open shared object file: No such file or directory

Round:  1
Round:  1
Round:  1
Round:  1
Round:  1
Round:  1
Round:  1
Round:  1
Round:  1
Round:  1
Round:  1
Round:  1
Round:  1
Round:  1
All Done~ ^^ muti-process try by leoqaz12


multiprocessing.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "<ipython-input-14-a992b6355e35>", line 48, in do_SBBGCA_clustering
    scoreList = findMergeCandidateScoreList(toMergeCandidateDict, generatedSeqNum)
  File "<ipython-input-14-d3a5a393b8a2>", line 26, in findMergeCandidateScoreList
    repNew = get_Representative(Ri, Rj)
  File "<ipython-input-14-107a0de4b2aa>", line 16, in get_Representative
    comMotif_dict = do_globalAlignment(rep1, rep2) # do Alignment
  File "<ipython-input-14-ba37b025efea>", line 21, in do_globalAlignment
    outputStage = OutputStage(stageMatrixResult, None, BASE, Motif_Obj)
NameError: name 'OutputStage' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/leoqaz12/anaconda3/lib/python3.6/multiprocessing/pool.py", l

In [3]:
# for k in dicts:
#     print(k)

ymeta
rhtl
black
gbot
linkury
krap
tibs
tufik
androidos
pcmega
games
outbrowse
midie
expirio
hackkms
bocj
klez
jeefo
smheist
pykspa
swrort
vbna
navipromo
hupigon
mabezat
buterat
gvance
bundlore
madang
vobfus
zvuzona
agdgc
banker
hybris
gepys
tltv
aroc
valhalla
daws
pcclient
mywebsearch
softpulse
runouce
chir
installbrain
peed
yakes
swizzor
midia
reveton
tenga
adgazelle
lolbot
sdld
poisonivy
screensaver
shell
urelas
jaik
gupboot
vbkrypt
padodor
hala
downloadassistant
bladabindi
staget
krypt
shellhook
sytro
stagol
ipz
symmi
looked
temonde
magania
wecod
buzy
plocust
hicrazyk
fraudpack
almanahe
addlyrics
squarenet
qukart
mepaow
soltern
vundo
shiz
jadtre
dapato
scarsi
cridex
somoto
elkern
obfuscated
heye
awdm
smshoax
androm
bublik
chindo
berbew
downloadadmin
rimecud
byfh
valla
renos
ramnit
ndow
dinwod
pasta
fileinfector
pornoasset
installrex
xtrat
dorkbot
wplugin
hllw
zbot
fiseria
scar
conjar
yontoo
softcnapp
vjadtre
vkopampi
sohanad
regrun
ursnif
xiazai
bayrob
linkular
beebone
vbran
imali


## 下面是multi-thread的main cell 方法
### 採用上面的multi-processing 這邊就別跑

In [8]:
class myThread (threading.Thread):
    def __init__(self, threadID, name, gen):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.gen = gen
    def run(self):
        print ("Starting: " + self.name,self.gen)
        # Get lock to synchronize threads
#         threadLock.acquire()
        muti_thread_RasMMA(self.name, self.gen)
        print ("Exiting: " + self.name,self.gen)
        # Free lock to release next thread
#         threadLock.release()

In [9]:
# basic global inputs variable
manualThresholdNumber = 0.8 # defined the threshold of merge score
# parameters
leo = ['elkern','eggnog','mira','autoit','parite','shodi','lydra'] #想要自動化完成的家族名稱
datadir='../data/train_17family/' #前綴路徑，family所在的根路徑
outpath = '../data/output/RasMMA-test/' #輸出資料夾根路徑
delay=35 #依照CPU單核心執行力自己調整，噴error就調大
threadNum=100 #CPU核心線程占用數量，delay越大可以設的越大，記憶體不夠者請注意此量

def muti_thread_RasMMA(familyName,generation):
    data_directory = datadir+ familyName +"/"+ generation +"/" # traces dir

    tag = familyName + "_0.8" # used for pickle name
    outputPath = outpath + tag + "/" + generation + "/"
    pickleDir = outputPath + "pickle/"

    # Create Directories if didn't exist

    if not os.path.isdir(outputPath): os.makedirs(outputPath)
    if not os.path.isdir(pickleDir): os.makedirs(pickleDir)

    # do clustering and output two pickle files. (@_intermediate.pickle and @_nameDict.pickle)
    % run RasMMA.ipynb

    def startClustering(data_directory, tag, outputPath, thresholdValue=None):
        if not os.path.isdir(outputPath): os.makedirs(outputPath) # create output path Directory

        intermediatePool, initialDict, roundInfos, residual = do_SBBGCA_clustering(data_directory, tag, outputPath, thresholdValue)

        # saving intermediatePool as pickle file
        with open(pickleDir + tag + '_intermediate.pickle', 'wb') as handle:
            pickle.dump(intermediatePool, handle, protocol=pickle.HIGHEST_PROTOCOL)

        # saving initialNames dict as pickle file
        with open(pickleDir + tag + '_initialDict.pickle', 'wb') as handle:
            pickle.dump(initialDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

        # saving round information dict as pickle file
        with open(pickleDir + tag + '_roundInfos.pickle', 'wb') as handle:
            pickle.dump(roundInfos, handle, protocol=pickle.HIGHEST_PROTOCOL)

        if(residual is not None):
            # saving round information dict as pickle file
            with open(pickleDir + tag + '_residual.pickle', 'wb') as handle:
                pickle.dump(residual, handle, protocol=pickle.HIGHEST_PROTOCOL)



    # usage example of get clustering results


#     date_time = datetime.datetime.now()
#     print(date_time.strftime("%Y-%b-%d %H:%M"))

    startClustering(data_directory, tag, outputPath, manualThresholdNumber)

#     date_time = datetime.datetime.now()
#     print(date_time.strftime("%Y-%b-%d %H:%M"))
    print('+++',pickleDir,'+++')
    
    
    
    
# threadLock = threading.Lock()
threads = []
c=0
for k in leo: #dicts
    familyName = k
#     print('===============',familyName,'=====================')
    for v in glob.glob(datadir+familyName+'/*'):
        generation = v.split('/')[-1]
#         print('-----------',familyName,generation,'--------------')
        thread1 = myThread(c, familyName, generation)
        thread1.start()
        threads.append(thread1)
        c+=1
        if c==threadNum: #達到線程數上限
            time.sleep(delay/10)
            print('==============Joinig threads firstly due to poor CPU and RAM...===============')
            for t in threads:
                t.join()
                if t in threads:
                    threads.remove(t)             
#             thread1.join() 
            c=0
        time.sleep(delay)  #防止奇怪error
#         thread2 = myThread(2, familyName, generation)
#         thread2.start()
#         thread3 = myThread(3, familyName, generation)
#         thread3.start()
#         thread4 = myThread(4, familyName, generation)
#         thread4.start()
# thread1.join()
print('=============Wait for Joining all other slow threads....QQ=================')
for t in threads:
    t.join()
print("All Done~ ^^ muti-thread try by leoqaz12")

Starting: elkern child
-- Finish Initializing --
+++ ../data/output/RasMMA-test/elkern_0.8/child/pickle/ +++
Exiting: elkern child
Starting: elkern main
-- Finish Initializing --
Starting: elkern 4 child
-- Finish Initializing --
+++ ../data/output/RasMMA-test/elkern_0.8/4 child/pickle/ +++
Exiting: elkern 4 child
Starting: elkern 3 child
-- Finish Initializing --
+++ ../data/output/RasMMA-test/elkern_0.8/3 child/pickle/ +++
Exiting: elkern 3 child
Starting: eggnog child
-- Finish Initializing --
+++ ../data/output/RasMMA-test/eggnog_0.8/child/pickle/ +++
Exiting: eggnog child
Starting: eggnog main
Starting: mira 5 child
-- Finish Initializing --
No common motif
+++ ../data/output/RasMMA-test/mira_0.8/5 child/pickle/ +++
Exiting: mira 5 child
Starting: mira child
-- Finish Initializing --
Starting: mira main
-- Finish Initializing --
Starting: mira 4 child
-- Finish Initializing --
+++ ../data/output/RasMMA-test/mira_0.8/4 child/pickle/ +++
Exiting: mira 4 child
Starting: mira 3 child


* REF: https://stackoverflow.com/questions/46247047/how-to-make-the-main-thread-wait-for-the-internal-threads-after-starting-them


In [124]:
# print(lists)
# del(list)
# list(dicts.keys())
# [dicts.keys()]
# ll=[]
# for k in dicts:
#     ll.append(k)
# ll
#'elkern','eggnog','mira','autoit','parite','shodi','lydra'

['madang',
 'soltern',
 'somoto',
 'elkern',
 'ramnit',
 'eggnog',
 'mira',
 'strictor',
 'ibryte',
 'autoit',
 'parite',
 'firseria',
 'graftor',
 'solimba',
 'shodi',
 'bettersurf',
 'lydra']

In [125]:
# list(dicts.keys())

['madang',
 'soltern',
 'somoto',
 'elkern',
 'ramnit',
 'eggnog',
 'mira',
 'strictor',
 'ibryte',
 'autoit',
 'parite',
 'firseria',
 'graftor',
 'solimba',
 'shodi',
 'bettersurf',
 'lydra']

# 以上Main part END
** revised by leoqaz12@2018/0414 **
* 自動化完成所指定的家族
* 增加顯示資訊，以追蹤進度
* 強迫多執行序，約可加速兩倍時間
***
if kernel dead at some generatedSeqNum,
##### Modify RasMMA.ipynb
def-do_SBBGCA_clustering() line30,31. and re_run all.

### Below cell used to write csv - _groupInfo, _decendants, _motifs
groupInfo.csv can see merge score.

###  multi-thread for speeding up
*revised by leoqaz12@2018/04/14*
* 修改為多執行序平行處理以加速(大概只提升10%吧...)
* 沿用main cell的leo、datadir、outpath參數

In [8]:
class myThread2 (threading.Thread):
    def __init__(self, threadID, name, gen):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.gen = gen
    def run(self):
        print ("Starting: " + self.name,self.gen)
        # Get lock to synchronize threads
#         threadLock.acquire()
        muti_thread_CSV(self.name, self.gen)
        print ("Exiting: " + self.name,self.gen)
        # Free lock to release next thread
#         threadLock.release()

In [9]:
def muti_thread_CSV(familyName,generation):
    data_directory = datadir+ familyName +"/"+ generation +"/" # traces dir

    tag = familyName + "_0.8" # used for pickle name
    outputPath = outpath + tag + "/" + generation + "/"
    pickleDir = outputPath + "pickle/"

    # read the results from pickle files
    with open(pickleDir + tag + '_intermediate.pickle', 'rb') as handle:
        intermediate = pickle.load(handle)
    with open(pickleDir + tag + '_initialDict.pickle', 'rb') as handle:
        initialDict = pickle.load(handle)
    with open(pickleDir + tag + '_roundInfos.pickle', 'rb') as handle:
        roundInfos = pickle.load(handle)

    # calculate motif lengths of all common motifs
    def getMotifsLengthList(motifs):
        motifLens = [] #list()
        for motif in motifs:
            startIdx =motif[1]
            endIdx = motif[2]
            mLen = endIdx - startIdx + 1
            motifLens.append(mLen)
        return motifLens

    def findGeneratedRoundNumber(clusterName, roundInfosDict):
        for key, value in roundInfosDict.items():
            if clusterName in value:
                return key
        return -1



    descendant_dict = dict()
    groupInfo_list = [] #list()
    groupMotif_dict = dict()

    intermediate_list = sorted(intermediate.items(), key=lambda x : x[0])
    for item in intermediate_list:
        value = item[1] # get original dict value
        score = value[0]
        clusterName = value[1][0]
        memberSet = value[2]
        motifs = value[1][1]

        # calculate motif lengths of all common motifs
        motifsLens = getMotifsLengthList(motifs) # is a list of numbers
        totalMotifLen = sum(motifsLens) # sum the list

        motifsCount = len(motifs)

        descendants = set()
        for member in memberSet:
            if member[0] == "G":
                for descendant in descendant_dict[member]:
                    descendants.add(descendant)
            else:
                descendants.add(member)

        descendant_dict[clusterName] = descendants


        groupMotif_dict[clusterName] = motifs
        roundNumber = findGeneratedRoundNumber(clusterName, roundInfos)
        groupInfo_list.append((roundNumber, clusterName, score, memberSet, motifsCount, motifsLens, totalMotifLen))

    with open(pickleDir + tag + "_descendant.pickle", 'wb') as f:
        pickle.dump(descendant_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

    # write file "GroupInfo.csv" :  clusterName, score, members, motifCount, common motifs length list
    with open(outputPath + tag + "_GroupInfo.csv", 'w', newline='') as infoFile:
        spamwriter = csv.writer(infoFile, delimiter=',',
                                quotechar='"', quoting=csv.QUOTE_MINIMAL)
        header = ["Round", "ClusterName", "SimilarityScore", "Members", "MotifsCount", "Motifs_Length", "Total_MotifLength"]
        spamwriter.writerow(header)

        # write initial cluster informations(i.e., hooklogs)
        for key in sorted(initialDict.keys(), key = lambda x : int(x[1::])):
            # something like this: (0, "G1", "N/A", "abc", 1, 109)
            originDataRow = (0, key, "N/A", initialDict[key][0], 1, initialDict[key][1], initialDict[key][1])
            spamwriter.writerow(originDataRow)

        # write cluster informations
        for group in groupInfo_list:
            spamwriter.writerow(group)

    with open(outputPath + tag + "_Descendants.csv", "w", newline='') as descFile:
        spamwriter = csv.writer(descFile, delimiter=',',
                                quotechar='"', quoting=csv.QUOTE_MINIMAL)
        header = ["ClusterName", "Descendant Counts", "Descendants"]
        spamwriter.writerow(header)
        for key in sorted(descendant_dict.keys(), key = lambda x : int(x[1::])):
            row = (key, len(descendant_dict[key]), descendant_dict[key])
            spamwriter.writerow(row)

    # write file "Motifs.csv" :  clusterName, MotifNumber, apis
    with open(outputPath + tag + "_Motifs.csv", 'w', newline='', encoding='utf-8') as motifFile:
        spamwriter = csv.writer(motifFile, delimiter=',',
                                quotechar='"', quoting=csv.QUOTE_MINIMAL)
        header = ["ClusterName", "MotifIndex", "MotifLength", "Common Motif APIs"]
        spamwriter.writerow(header)

        for key in sorted(groupMotif_dict.keys(), key = lambda x : int(x[1::])):
            group_motifs = groupMotif_dict[key]
            motifIdx = 0
            for motif in group_motifs:
                firstMotifAPI = True
                motifLen = len(motif[0])
                for api in motif[0]:
                    if(firstMotifAPI):
                        row = (key, motifIdx, motifLen, api)
                        firstMotifAPI = False
                    else:
                        row = ("", "", "", api)
                    spamwriter.writerow(row)
                motifIdx += 1

    # output residual information of SBBGCA

    with open(pickleDir + tag + '_residual.pickle', 'rb') as handle:
        residual = pickle.load(handle)

    with open(outputPath + tag + "_GroupInfo.csv", 'a', newline='') as expandGroupInfo:
        spamwriter = csv.writer(expandGroupInfo, delimiter=',',
                                quotechar='"', quoting=csv.QUOTE_MINIMAL)
        header = ["ClusterName", "Members", "MotifLength"]

        spamwriter.writerow("")
        spamwriter.writerow(("Residual Clusters:","",""))
        spamwriter.writerow(header)

        for key, value in residual.items():
            clusterName = value[0][0]
            motifsList = value[0][1]
            motifLens = getMotifsLengthList(motifsList)
            members = value[1]
            if( len(members) == 0 ):
                row = (clusterName, "N/A", motifLens)
            else:
                row = (clusterName, members, motifLens)

            spamwriter.writerow(row)

threads = []
c=0
for k in dicts: #leo
    familyName = k
#     print('===============',familyName,'=====================')
    for v in glob.glob(datadir+familyName+'/*'):
        generation = v.split('/')[-1]
#         print('-----------',familyName,generation,'--------------')
        thread1 = myThread2(1, familyName, generation)
        thread1.start()
        threads.append(thread1)
        c+=1
        if c==threadNum: #達到線程數上限
            time.sleep(delay/10000)
            print('==============Joinig threads firstly due to poor CPU and RAM...===============')
            for t in threads:
                t.join()
                if t in threads:
                    threads.remove(t)             
#             thread1.join() 
            c=0
        time.sleep(delay/1000)  #防止奇怪error
#         thread2 = myThread(2, familyName, generation)
#         thread2.start()
#         thread3 = myThread(3, familyName, generation)
#         thread3.start()
#         thread4 = myThread(4, familyName, generation)
#         thread4.start()
# thread1.join()
print('=============Wait for Joining all other slow threads....QQ=================')
for t in threads:
    t.join()
print("All CSV Done~ ^^")

NameError: name 'datadir' is not defined

***

### Below cell used for drawing trees.

In [10]:
# draw pics

def drawClusteringResults(picklePath, outputPath, tag, upgma_threshold):
    
    #     Dependencies
    import pickle
    import scipy
    import scipy.cluster.hierarchy as sch
    import matplotlib.pylab as plt
    %matplotlib inline
    
    # read the results from pickle files
    with open(picklePath + tag + '_intermediate.pickle', 'rb') as handle:
        intermediate = pickle.load(handle)
    with open(picklePath + tag + '_initialDict.pickle', 'rb') as handle:
        initialDict = pickle.load(handle)
    with open(picklePath + tag + '_roundInfos.pickle', 'rb') as handle:
        roundInfos = pickle.load(handle)
        
    initialNameDict = getInitialNameDict(initialDict)
    print("Original Names : ", initialNameDict)
    print("round informations", roundInfos)
    
    # It have to create the Z structure for drawing purpose.
    Z = createStructZ(intermediate, initialNameDict)
    
    label_list = createLabelList(initialNameDict) # create graph labels by nameDict

    (orig_x, orig_y) = plt.rcParams['figure.figsize']
    plt.rcParams['figure.figsize'] = (6, 10) #---input

    # P = sch.dendrogram(Z, color_threshold = upgma_threshold, orientation = 'right') # no label
    P = sch.dendrogram(Z, color_threshold = upgma_threshold, labels = label_list, orientation = 'right')

#     plt.axvline(x=upgma_threshold, linewidth=1, color='black', linestyle='--')
    locs, labels = plt.yticks()
    # plt.xticks(  np.arange(0,1.1,0.1)) #---Align axis-x 900(0, 0.35, 0.05) 909(0,0.6,0.1) 855(0, 0.9, 0.1)
    plt.setp(labels, fontsize = 14)
    plt.tight_layout()

    plt.rcParams['figure.figsize'] = (orig_x, orig_y)
    plt.savefig(outputPath+'SBBGCA_'+tag+'.pdf', dpi=300)

In [11]:
# usage example of draw hierarchy graph of clustering results

drawClusteringResults(pickleDir, outputPath, tag, 0.01)

Original Names :  {'G12': '2e4850_3368', 'G39': 'ab9903_3220', 'G47': 'bfefab_3212', 'G6': '1192d4_3372', 'G14': '382d34_3292', 'G41': 'ac6a42_3284', 'G49': 'c9955c_3040', 'G2': '077940_3264', 'G26': '817033_2896', 'G36': '9fd1e8_3116', 'G40': 'ac6a42_1312', 'G45': 'bf6239_3268', 'G56': 'd5e1d2_3308', 'G25': '78ac49_3284', 'G9': '24d255_3264', 'G55': 'd3e023_3364', 'G31': '8bcf66_2916', 'G0': '008e76_3272', 'G8': '20a21e_3332', 'G44': 'bf6239_3212', 'G46': 'bfefab_3196', 'G37': 'a4d38d_3316', 'G32': '8e2bca_2992', 'G50': 'caecca_3200', 'G28': '85ed58_3356', 'G52': 'cfc485_2996', 'G27': '841bfb_3296', 'G18': '460dad_3324', 'G22': '63117f_3260', 'G42': 'ad7615_2928', 'G5': '1192d4_3296', 'G21': '524d5d_2992', 'G53': 'cfc485_3328', 'G35': '9b791b_3232', 'G19': '4a6eb4_2972', 'G23': '751eba_3328', 'G54': 'cfd1ac_3296', 'G17': '456ced_3316', 'G30': '897e05_2916', 'G7': '121340_2924', 'G57': 'dc813f_3288', 'G51': 'cf5d72_3320', 'G59': 'e64f40_3280', 'G38': 'a56c48_2920', 'G11': '2e4850_3284'

ValueError: Linkage 'Z' uses non-singleton cluster before it is formed.